# Praca domowa numer 4

## zadanie 0 - wczytanie funkcji

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import tempfile
import os
import sqlite3

In [2]:
Badges    = pd.read_csv('/home/kuba/Documents/Dane/Badges.csv')
Comments  = pd.read_csv('/home/kuba/Documents/Dane/Comments.csv')
PostLinks = pd.read_csv('/home/kuba/Documents/Dane/PostLinks.csv')
Posts     = pd.read_csv('/home/kuba/Documents/Dane/Posts.csv')
Tags      = pd.read_csv('/home/kuba/Documents/Dane/Tags.csv')
Users     = pd.read_csv('/home/kuba/Documents/Dane/Users.csv')
Votes     = pd.read_csv('/home/kuba/Documents/Dane/Votes.csv')

baza = os.path.join(tempfile.mkdtemp(), "baza.db")

if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn) 
PostLinks.to_sql("PostLinks", conn) 
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn) 
Users.to_sql("Users", conn) 
Votes.to_sql("Votes", conn) 


## Zadanie 1 

-**SQL**

In [14]:
df1_sql = pd.read_sql_query("""
SELECT
Users.DisplayName,
Users.Age,
Users.Location,
SUM(Posts.FavoriteCount) AS FavoriteTotal,
Posts.Title AS MostFavoriteQuestion,
MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)

-**Pandas**

In [15]:
# łączę obie tabelki w jedą
df = pd.merge(Users, Posts, left_on='Id', right_on='OwnerUserId')

In [16]:
#Wybieram te kolumny, z którymi będę coś jeszcze robił
df1 = df[['DisplayName', 'Age', 'Location', 'FavoriteCount', 'Title','OwnerUserId', 'PostTypeId']]

In [17]:
# grupuję
df2 = df1.groupby('OwnerUserId')

# wykonuję operacje na grupach
df3 = df2.FavoriteCount.agg([sum, max]).merge(df1,
                                              left_on=['OwnerUserId','max'], 
                                              right_on = ['OwnerUserId','FavoriteCount']).rename(
    columns = {  'sum':'FavoriteTotal',
                 'max':'MostFavoriteQuestionLikes',
                 'Title':'MostFavoriteQuestion'})


In [19]:
# filtruję i sortuję, biorę 10 pierszych i resetuję index
df4 = df3[df3.PostTypeId == 1].sort_values('FavoriteTotal', ascending=False).head(10).reset_index()

# wybieram te kolumny co w zapytaniu sql
df5 = df4[['DisplayName','Age', 'Location','FavoriteTotal','MostFavoriteQuestion',"MostFavoriteQuestionLikes" ]]

# zamieniam Nan na None
df5.Location = df5.Location.where(pd.notnull(df5.Location), None)

df_pandas_1 = df5

In [20]:
#sprawdźmy czy wyszło nam to samo 
df_pandas_1.equals(df1_sql)

True

# Zadanie 2

-**SQL**

In [100]:
df2_sql = pd.read_sql_query("""SELECT
Posts.ID,
Posts.Title,
Posts2.PositiveAnswerCount
FROM Posts
JOIN (
SELECT
Posts.ParentID,
COUNT(*) AS PositiveAnswerCount
FROM Posts
WHERE Posts.PostTypeID=2 AND Posts.Score>0
GROUP BY Posts.ParentID
) AS Posts2
ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10""", conn)

-**Pandas*** 

In [105]:
Posts2 = Posts.loc[(Posts.PostTypeId == 2) & (Posts.Score >0 )]

df2 =    Posts2.groupby('ParentId').size().to_frame().rename( columns = {0 : 'PositiveAnswerCount'})

df3 =    df2.merge( Posts , left_on = 'ParentId', right_on = 'Id')[[
            'Id','Title', 'PositiveAnswerCount']].reset_index().sort_values([
            'PositiveAnswerCount', 'Id'],ascending = [False,True]).reset_index()[['Id','Title','PositiveAnswerCount']].head(10)


In [106]:
df2_sql

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


In [108]:
df3.equals(df2_sql)

True

## Zadanie 3

-**SQL**

In [206]:
df3_sql = pd.read_sql_query("""SELECT
Posts.Title,
UpVotesPerYear.Year,
MAX(UpVotesPerYear.Count) AS Count
FROM (
SELECT
PostId,
COUNT(*) AS Count,
STRFTIME('%Y', Votes.CreationDate) AS Year
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, Year
) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year""", conn)

In [207]:
df1 = Votes.loc[Votes.VoteTypeId == 2]
df1.CreationDate = df1.CreationDate.str[:4]
df1 = df1.rename(columns = {'CreationDate':'Year'})

UpvotesPerYear = df1.groupby(['PostId', 'Year']).size().to_frame('Count').reset_index()

df2 = pd.merge(UpvotesPerYear, Posts, left_on='PostId', right_on='Id')
df2 = df2.loc[df2.PostTypeId == 1]

df3 = df2.groupby('Year').Count.agg(max).to_frame('Count').reset_index().merge(df2, on=['Year', 'Count'])

df4 = df3[['Title','Year', 'Count']]

In [208]:
df4.equals(df3_sql)

True

## Zadanie 4 

-**SQL** 

In [226]:
df4_sql = pd.read_sql_query("""SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT * FROM Posts
WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC""", conn)

In [278]:
BestAnswers =    Posts.loc[Posts.PostTypeId == 2].groupby(
                'ParentId').Score.agg(max).to_frame('MaxScore').reset_index().merge(
                Posts, left_on=['ParentId', 'MaxScore'], right_on=['ParentId', 'Score'])

BestAnswers = BestAnswers[['Id','ParentId', 'MaxScore']]

Questions = Posts.loc[Posts.PostTypeId == 1]

df1 = pd.merge(Questions, BestAnswers, left_on='Id', right_on = 'ParentId'
              ).rename(columns = {'Score':'QScore'})

df2 = pd.merge(df1, Posts, left_on = 'AcceptedAnswerId', right_on='Id').reset_index(
        )[['Id_x','Title_x','Score','MaxScore']]

df2['Difference'] = df2.MaxScore - df2.Score
df2
df3 = df2.sort_values('Difference', ascending = False).reset_index()[[
       'Id_x','Title_x','MaxScore', 'Score','Difference']].rename(
                                                         columns = {'Score':'AcceptedScore',
                                                                    'Id_x':'Id',
                                                                    'Title_x':'Title'})

df4 = df3.loc[df3.Difference > 50]

In [281]:
df4.equals(df4_sql)

True

## Zadanie 5 

-**SQL** 

In [284]:
df5_sql = pd.read_sql_query("""SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10""", conn)

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


In [344]:
CmtTotScr = Comments.groupby(['PostId','UserId']).Score.agg(sum).to_frame('CommentsTotalScore').reset_index()

df1 = pd.merge(CmtTotScr, Posts, left_on=['PostId','UserId'], right_on=['Id','OwnerUserId'])

df2 = df1.loc[df1.PostTypeId == 1].sort_values(
    'CommentsTotalScore',ascending = False).reset_index().sort_values(
    ['CommentsTotalScore','index'], ascending=[False,True]).reset_index()[['Title','CommentsTotalScore']].head(10)

In [346]:
df2.equals(df5_sql)

True

## Zadanie 6 

-**SQL** 

In [349]:
df6_sql = pd.read_sql_query("""SELECT
DISTINCT
Users.Id,
Users.DisplayName,
Users.Reputation,
Users.Age,
Users.Location
FROM (
SELECT
Name, UserID
FROM Badges
WHERE Name IN (
SELECT
Name
FROM Badges
WHERE Class=1
GROUP BY Name
HAVING COUNT(*) BETWEEN 2 AND 10
)
AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id""", conn)

-**Pandas**

In [382]:
df1 = Badges.loc[Badges.Class == 1].groupby('Name').size().to_frame('Count').reset_index()

df2 = df1.loc[(df1.Count >=2) & (df1.Count <= 10) ][['Name']]

ValuableBadges = Badges.loc[Badges.Name.isin(df2.Name)].loc[Badges.Class == 1][['Name','UserId']]

df3 = pd.merge(ValuableBadges, Users, left_on = 'UserId', right_on='Id')[['Id','DisplayName','Reputation','Age',
                                                                          'Location']].drop_duplicates(
                                                                            ).reset_index(drop=True)

In [386]:
df3.equals(df6_sql)

True

## Zadanie 7 

-**SQL** 

In [389]:
df7_sql = pd.read_sql_query("""SELECT
Posts.Title,
VotesByAge2.OldVotes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2017' THEN 'new'
WHEN '2016' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, VoteDate
) AS VotesByAge
GROUP BY VotesByAge.PostId
HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10""", conn)

-**Pandas**

In [469]:
df1 = Votes.loc[Votes.VoteTypeId == 2]

df1.CreationDate = df1.CreationDate.str[:4]

df1.CreationDate = ['new' if e in ['2016','2017'] else 'old' for e in df1.CreationDate]
df2 = df1.rename(columns = {'CreationDate':'VoteDate'})

VotesByAge = df2.groupby(['PostId','VoteDate']).size().unstack(fill_value=0).stack().to_frame('Total').reset_index()

#NewVotes = VotesByAge2[VotesByAge2.VoteDate == 'new'][['Total']]

temp1 = VotesByAge.loc[VotesByAge.VoteDate == 'new'].groupby('PostId').Total.agg(max).to_frame(
                                                                'NewVotes').reset_index()

temp2 = VotesByAge.loc[VotesByAge.VoteDate == 'old'].groupby('PostId').Total.agg(max).to_frame(
                                                                'OldVotes').reset_index()

VotesByAge2 = pd.merge(temp1, temp2, on='PostId')
VotesByAge2 = VotesByAge2.loc[VotesByAge2.NewVotes == 0]

df3 = pd.merge(VotesByAge2, Posts , left_on = 'PostId', right_on = 'Id')

df4 = df3.loc[df3.PostTypeId == 1][['Title', 'OldVotes']].sort_values('OldVotes', ascending = False).reset_index(
                                    ).sort_values(['OldVotes','index'], ascending = [False, True])[[
                                    'Title', 'OldVotes']].head(10).reset_index(drop=True)


,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


In [471]:
df4.equals(df7_sql)

True